In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

In [2]:
def n(digits=3):
    number = ''
    for i in range(np.random.randint(1, digits + 1)):
        number += np.random.choice(list('0123456789'))
    return int(number)

In [3]:
def padding(chars, maxlen):
    return chars + ' ' * (maxlen - len(chars))

In [4]:
N = 20000
N_train = int(N * 0.9)
N_validation = N - N_train

digits = 3
input_digits = digits * 2 + 1
output_digits = digits + 1

added = set()
questions = []
answers = []

while len(questions) < N:
    a, b = n(), n()
    
    pair = tuple(sorted((a, b)))
    if pair in added:
        continue
        
    question = '{}+{}'.format(a, b)
    question = padding(question, input_digits)
    answer = str(a + b)
    answer = padding(answer, output_digits)
    
    added.add(pair)
    questions.append(question)
    answers.append(answer)

In [5]:
chars = '0123456789+ '
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

In [6]:
X = np.zeros((len(questions), input_digits, len(chars)), dtype=np.integer)
Y = np.zeros((len(questions), digits + 1, len(chars)), dtype=np.integer)

for i in range(N):
    for t, char in enumerate(questions[i]):
        X[i, t, char_indices[char]] = 1
    for t, char in enumerate(answers[i]):
        Y[i, t, char_indices[char]] = 1

X_train, X_validation, Y_train, Y_validation = train_test_split(X, Y, train_size=N_train)

In [7]:
def inference(x, y, n_batch, is_training,
              input_digits=None, output_digits=None,
              n_hidden=None, n_out=None):
    def weight_variable(shape):
        initial = tf.truncated_normal(shape, stddev=0.01)
        return tf.Variable(initial)

    def bias_variable(shape):
        initial = tf.zeros(shape, dtype=tf.float32)
        return tf.Variable(initial)

    # Encoder
    encoder = tf.contrib.rnn.BasicLSTMCell(n_hidden, forget_bias=1.0)
    state = encoder.zero_state(n_batch, tf.float32)
    encoder_outputs = []
    encoder_states = []

    with tf.variable_scope('Encoder'):
        for t in range(input_digits):
            if t > 0:
                tf.get_variable_scope().reuse_variables()
            (output, state) = encoder(x[:, t, :], state)
            encoder_outputs.append(output)
            encoder_states.append(state)

    # Decoder
    decoder = tf.contrib.rnn.BasicLSTMCell(n_hidden, forget_bias=1.0)
    state = encoder_states[-1]
    decoder_outputs = [encoder_outputs[-1]]

    # 出力層の重みとバイアスを事前に定義
    V = weight_variable([n_hidden, n_out])
    c = bias_variable([n_out])
    outputs = []

    with tf.variable_scope('Decoder'):
        for t in range(1, output_digits):
            if t > 1:
                tf.get_variable_scope().reuse_variables()

            if is_training is True:
                (output, state) = decoder(y[:, t-1, :], state)
            else:
                # 直前の出力を入力に用いる
                linear = tf.matmul(decoder_outputs[-1], V) + c
                out = tf.nn.softmax(linear)
                outputs.append(out)
                out = tf.one_hot(tf.argmax(out, -1), depth=output_digits)
                (output, state) = decoder(out, state)

            decoder_outputs.append(output)

    if is_training is True:
        output = tf.reshape(tf.concat(decoder_outputs, axis=1),
                            [-1, output_digits, n_hidden])

        linear = tf.einsum('ijk,kl->ijl', output, V) + c
        # linear = tf.matmul(output, V) + c
        return tf.nn.softmax(linear)
    else:
        # 最後の出力を求める
        linear = tf.matmul(decoder_outputs[-1], V) + c
        out = tf.nn.softmax(linear)
        outputs.append(out)

        output = tf.reshape(tf.concat(outputs, axis=1),
                            [-1, output_digits, n_out])
        return output

In [8]:
def loss(y, t):
    return tf.reduce_mean(tf.square(y-t))

def training(loss):
    optimizer = tf.train.AdamOptimizer(learning_rate=0.001, beta1=0.9, beta2=0.999)
    
    train_step = optimizer.minimize(loss)
    return train_step

In [9]:
def accuracy(y, t):
    correct_prediction = tf.equal(tf.argmax(y, -1), tf.argmax(t, -1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    return accuracy

In [10]:
n_in = len(chars)
n_hidden = 128
n_out = len(chars)

x = tf.placeholder(tf.float32, shape=[None, input_digits, n_in])
t = tf.placeholder(tf.float32, shape=[None, output_digits, n_out])
n_batch = tf.placeholder(tf.int32, shape=[])
is_training = tf.placeholder(tf.bool)

y = inference(x, t, n_batch, is_training,
                  input_digits=input_digits,
                  output_digits=output_digits,
                  n_hidden=n_hidden, n_out=n_out)
loss = loss(y, t)
train_step = training(loss)

acc = accuracy(y, t)
history = {
        'val_loss': [],
        'val_acc': []
    }

In [11]:
epochs = 200
batch_size = 200

init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

n_batches = N_train // batch_size

In [12]:
for epoch in range(epochs):
    print('=' * 10)
    print('Epoch:', epoch)
    print('=' * 10)

    X_, Y_ = shuffle(X_train, Y_train)

    for i in range(n_batches):
        start = i * batch_size
        end = start + batch_size

        sess.run(train_step, feed_dict={
            x: X_[start:end],
            t: Y_[start:end],
            n_batch: batch_size,
            is_training: True
        })

    # 検証データを用いた評価
    val_loss = loss.eval(session=sess, feed_dict={
        x: X_validation,
        t: Y_validation,
        n_batch: N_validation,
        is_training: False
    })
    val_acc = acc.eval(session=sess, feed_dict={
        x: X_validation,
        t: Y_validation,
        n_batch: N_validation,
        is_training: False
    })

    history['val_loss'].append(val_loss)
    history['val_acc'].append(val_acc)
    print('validation loss:', val_loss)
    print('validation acc: ', val_acc)
    
    if epoch % 50 == 0:
        # 検証データからランダムに問題を選んで答え合わせ
        for i in range(10):
            index = np.random.randint(0, N_validation)
            question = X_validation[np.array([index])]
            answer = Y_validation[np.array([index])]
            prediction = y.eval(session=sess, feed_dict={
                x: question,
                # t: answer,
                n_batch: 1,
                is_training: False
            })
            question = question.argmax(axis=-1)
            answer = answer.argmax(axis=-1)
            prediction = np.argmax(prediction, -1)

            q = ''.join(indices_char[i] for i in question[0])
            a = ''.join(indices_char[i] for i in answer[0])
            p = ''.join(indices_char[i] for i in prediction[0])

            print('-' * 10)
            print('Q:  ', q)
            print('A:  ', p)
            print('T/F:', end=' ')
            if a == p:
                print('T')
            else:
                print('F')
        print('-' * 10)

Epoch: 0
validation loss: 0.058947
validation acc:  0.34775
----------
Q:   31+55  
A:   11  
T/F: F
----------
Q:   45+21  
A:   11  
T/F: F
----------
Q:   17+351 
A:   11  
T/F: F
----------
Q:   319+237
A:   111 
T/F: F
----------
Q:   984+2  
A:   11  
T/F: F
----------
Q:   23+46  
A:   11  
T/F: F
----------
Q:   0+522  
A:   11  
T/F: F
----------
Q:   58+15  
A:   11  
T/F: F
----------
Q:   609+61 
A:   111 
T/F: F
----------
Q:   155+560
A:   111 
T/F: F
----------
Epoch: 1
validation loss: 0.0583942
validation acc:  0.348
Epoch: 2
validation loss: 0.0580695
validation acc:  0.3515
Epoch: 3
validation loss: 0.057775
validation acc:  0.352875
Epoch: 4
validation loss: 0.0575561
validation acc:  0.355625
Epoch: 5
validation loss: 0.0574111
validation acc:  0.358125
Epoch: 6
validation loss: 0.0563534
validation acc:  0.362625
Epoch: 7
validation loss: 0.0553686
validation acc:  0.374
Epoch: 8
validation loss: 0.0541713
validation acc:  0.394625
Epoch: 9
validation loss: 0.0528

validation loss: 0.0277842
validation acc:  0.7295
Epoch: 89
validation loss: 0.0274734
validation acc:  0.736625
Epoch: 90
validation loss: 0.0272004
validation acc:  0.739375
Epoch: 91
validation loss: 0.0271017
validation acc:  0.740625
Epoch: 92
validation loss: 0.0268187
validation acc:  0.744125
Epoch: 93
validation loss: 0.0268385
validation acc:  0.745125
Epoch: 94
validation loss: 0.0266483
validation acc:  0.746875
Epoch: 95
validation loss: 0.0267023
validation acc:  0.748375
Epoch: 96
validation loss: 0.0263825
validation acc:  0.751625
Epoch: 97
validation loss: 0.0265069
validation acc:  0.75175
Epoch: 98
validation loss: 0.0263362
validation acc:  0.74975
Epoch: 99
validation loss: 0.0265431
validation acc:  0.752
Epoch: 100
validation loss: 0.0259503
validation acc:  0.754375
----------
Q:   5+607  
A:   612 
T/F: T
----------
Q:   543+5  
A:   548 
T/F: T
----------
Q:   59+748 
A:   707 
T/F: F
----------
Q:   97+92  
A:   180 
T/F: F
----------
Q:   334+5  
A:   339 

validation loss: 0.0236581
validation acc:  0.80675
Epoch: 176
validation loss: 0.024161
validation acc:  0.804
Epoch: 177
validation loss: 0.0229335
validation acc:  0.81375
Epoch: 178
validation loss: 0.0227543
validation acc:  0.816
Epoch: 179
validation loss: 0.0224704
validation acc:  0.81825
Epoch: 180
validation loss: 0.0230252
validation acc:  0.814375
Epoch: 181
validation loss: 0.0226571
validation acc:  0.81875
Epoch: 182
validation loss: 0.0225291
validation acc:  0.81925
Epoch: 183
validation loss: 0.0226058
validation acc:  0.81475
Epoch: 184
validation loss: 0.022736
validation acc:  0.817
Epoch: 185
validation loss: 0.0225877
validation acc:  0.818125
Epoch: 186
validation loss: 0.0225642
validation acc:  0.818875
Epoch: 187
validation loss: 0.0228212
validation acc:  0.816125
Epoch: 188
validation loss: 0.0235713
validation acc:  0.812
Epoch: 189
validation loss: 0.0250364
validation acc:  0.7975
Epoch: 190
validation loss: 0.0231821
validation acc:  0.815625
Epoch: 19

In [13]:
# 検証データからランダムに問題を選んで答え合わせ
for i in range(10):
    index = np.random.randint(0, N_validation)
    question = X_validation[np.array([index])]
    answer = Y_validation[np.array([index])]
    prediction = y.eval(session=sess, feed_dict={
        x: question,
        # t: answer,
        n_batch: 1,
        is_training: False
    })
    question = question.argmax(axis=-1)
    answer = answer.argmax(axis=-1)
    prediction = np.argmax(prediction, -1)

    q = ''.join(indices_char[i] for i in question[0])
    a = ''.join(indices_char[i] for i in answer[0])
    p = ''.join(indices_char[i] for i in prediction[0])

    print('-' * 10)
    print('Q:  ', q)
    print('A:  ', p)
    print('T/F:', end=' ')
    if a == p:
        print('T')
    else:
        print('F')
print('-' * 10)

----------
Q:   8+141  
A:   149 
T/F: T
----------
Q:   3+273  
A:   276 
T/F: T
----------
Q:   216+920
A:   1187
T/F: F
----------
Q:   61+857 
A:   919 
T/F: F
----------
Q:   2+918  
A:   920 
T/F: T
----------
Q:   2+376  
A:   378 
T/F: T
----------
Q:   794+683
A:   1590
T/F: F
----------
Q:   969+38 
A:   1005
T/F: F
----------
Q:   758+614
A:   1366
T/F: F
----------
Q:   837+60 
A:   816 
T/F: F
----------


# Kerasによる実装

In [14]:
from keras.optimizers import Adam
from keras.layers import Activation, Dense
from keras.layers.core import RepeatVector
from keras.layers.recurrent import LSTM
from keras.layers.wrappers import TimeDistributed
from keras.models import Sequential

Using TensorFlow backend.


In [15]:
model = Sequential()

# Encoder
model.add(LSTM(n_hidden, input_shape=(input_digits, n_in)))

# Decoder
model.add(RepeatVector(output_digits))
model.add(LSTM(n_hidden, return_sequences=True))
model.add(TimeDistributed(Dense(n_out)))
model.add(Activation('softmax'))
model.compile(loss='categorical_crossentropy',
             optimizer=Adam(lr=0.001, beta_1=0.9, beta_2=0.999),
             metrics=['accuracy'])

In [16]:
epochs = 200
batch_size = 200

for epoch in range(epochs):
    model.fit(X_train, Y_train, batch_size=batch_size, epochs=1,
              validation_data=(X_validation, Y_validation))

    if epoch % 50 == 0:
    # 検証データからランダムに問題を選んで答え合わせ
        for i in range(10):
            index = np.random.randint(0, N_validation)
            question = X_validation[np.array([index])]
            answer = Y_validation[np.array([index])]
            prediction = model.predict_classes(question, verbose=0)

            question = question.argmax(axis=-1)
            answer = answer.argmax(axis=-1)

            q = ''.join(indices_char[i] for i in question[0])
            a = ''.join(indices_char[i] for i in answer[0])
            p = ''.join(indices_char[i] for i in prediction[0])

            print('-' * 10)
            print('Q:  ', q)
            print('A:  ', p)
            print('T/F:', end=' ')
            if a == p:
                print('T')
            else:
                print('F')
        print('-' * 10)

Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 5s 284us/step - loss: 2.0372 - acc: 0.2926 - val_loss: 1.8210 - val_acc: 0.3499
----------
Q:   323+109
A:   112 
T/F: F
----------
Q:   777+13 
A:   112 
T/F: F
----------
Q:   260+8  
A:   11  
T/F: F
----------
Q:   818+570
A:   112 
T/F: F
----------
Q:   206+47 
A:   112 
T/F: F
----------
Q:   595+960
A:   1112
T/F: F
----------
Q:   17+258 
A:   112 
T/F: F
----------
Q:   167+908
A:   112 
T/F: F
----------
Q:   854+581
A:   112 
T/F: F
----------
Q:   30+334 
A:   112 
T/F: F
----------
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 5s 255us/step - loss: 1.7929 - acc: 0.3523 - val_loss: 1.7913 - val_acc: 0.3539
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 5s 253us/step - loss: 1.7674 - acc: 0.3583 - val_loss: 1.7649 - val_acc: 0.3561
Train on 18000 sample

Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 5s 255us/step - loss: 0.2782 - acc: 0.9242 - val_loss: 0.3416 - val_acc: 0.8869
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 5s 254us/step - loss: 0.2647 - acc: 0.9281 - val_loss: 0.3200 - val_acc: 0.8961
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 5s 255us/step - loss: 0.2498 - acc: 0.9326 - val_loss: 0.3363 - val_acc: 0.8831
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 5s 252us/step - loss: 0.2348 - acc: 0.9383 - val_loss: 0.3113 - val_acc: 0.8992
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 5s 254us/step - loss: 0.2291 - acc: 0.9374 - val_loss: 0.2983 - val_acc: 0.9007
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000

18000/18000 [==============================] - 5s 256us/step - loss: 0.0319 - acc: 0.9981 - val_loss: 0.1323 - val_acc: 0.9553
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 5s 253us/step - loss: 0.0306 - acc: 0.9982 - val_loss: 0.1305 - val_acc: 0.9544
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 5s 256us/step - loss: 0.0300 - acc: 0.9983 - val_loss: 0.1333 - val_acc: 0.9515
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 5s 258us/step - loss: 0.0278 - acc: 0.9988 - val_loss: 0.1296 - val_acc: 0.9552
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 5s 255us/step - loss: 0.0261 - acc: 0.9990 - val_loss: 0.1252 - val_acc: 0.9586
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 5s 256us/step - loss: 0

18000/18000 [==============================] - 5s 256us/step - loss: 0.0247 - acc: 0.9952 - val_loss: 0.1337 - val_acc: 0.9554
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 5s 255us/step - loss: 0.0103 - acc: 0.9998 - val_loss: 0.1210 - val_acc: 0.9602
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 5s 256us/step - loss: 0.0075 - acc: 1.0000 - val_loss: 0.1199 - val_acc: 0.9612
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 5s 256us/step - loss: 0.0065 - acc: 1.0000 - val_loss: 0.1186 - val_acc: 0.9627
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 5s 258us/step - loss: 0.0059 - acc: 1.0000 - val_loss: 0.1199 - val_acc: 0.9618
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 5s 258us/step - loss: 0

18000/18000 [==============================] - 5s 255us/step - loss: 0.0035 - acc: 1.0000 - val_loss: 0.1156 - val_acc: 0.9669
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 5s 255us/step - loss: 0.0032 - acc: 1.0000 - val_loss: 0.1148 - val_acc: 0.9665
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 5s 259us/step - loss: 0.0030 - acc: 1.0000 - val_loss: 0.1171 - val_acc: 0.9666
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 5s 255us/step - loss: 0.0028 - acc: 1.0000 - val_loss: 0.1157 - val_acc: 0.9665
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 5s 253us/step - loss: 0.0026 - acc: 1.0000 - val_loss: 0.1157 - val_acc: 0.9676
Train on 18000 samples, validate on 2000 samples
Epoch 1/1
18000/18000 [==============================] - 5s 255us/step - loss: 0